In [8]:
suppressPackageStartupMessages({
    library(ggplot2)
    library(ggpubr)
})

path <- "./"

In [9]:
species <- c("Hsap_lung", "Hsap_eye")

get_OR <- function(species, type){
    df <- Reduce(rbind, lapply(species, FUN = function(s){
        wgd <- read.delim(paste0(path, s, "/", s, ".ohnolog_DEGs.fisher.celltype.txt"), header = T)
        ssd <- read.delim(paste0(path, s, "/", s, ".SSDparalog_DEGs.fisher.celltype.txt"), header = T)
        
        wgd$type <- "WGD"
        ssd$type <- "SSD"
    
        OR_info <- rbind(wgd, ssd)
        OR_info$species <- s
        return(OR_info)
    }))
    my_comparisons <- list(c("WGD", "SSD"))
    p <- ggboxplot(df, x = "type", y = "OR", color = "type", palette = "jco") + 
        stat_compare_means(comparisons = my_comparisons, method = "wilcox.test", paired = TRUE, label = "p.signif") + 
        stat_summary(fun = "median", geom = "text", aes(label = round(after_stat(y), 3)), vjust = -1) + 
        facet_wrap(~species, nrow = 1) + 
        theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    ggsave(filename = paste0("OR.cell_family.summary.", type, ".pdf"), p, width = 4, height = 7)
}


In [10]:
get_OR(species, "wilcox")

In [11]:
species <- c("Hsap_lung", "Hsap_eye")

get_ratio <- function(species, type){
    df <- Reduce(rbind, lapply(species, FUN = function(s){
        wgd <- read.delim(paste0(path, s, "/", s, ".ohnolog_ratio_inDEGs.stats.txt"), header = T)
        ssd <- read.delim(paste0(path, s, "/", s, ".SSDparalog_ratio_inDEGs.stats.txt"), header = T)
    
        wgd$type <- "WGD"
        ssd$type <- "SSD"
        colnames(wgd) <- colnames(ssd)
    
        info <- rbind(wgd, ssd)
        info$species <- s
        return(info)
    
    }))

    
    ratio_bg <- Reduce(rbind, lapply(species, FUN = function(s){
        tmp <- read.delim(paste0(path, s, "/", s, ".ratio_bg.txt"), header = F)
        tmp <- tmp[tmp$V1 %in% c("ohnologs", "SSDparalogs"), ]
        tmp[tmp$V1 == "SSDparalogs", "V1"] <- "SSD"
        tmp[tmp$V1 == "ohnologs", "V1"] <- "WGD"
        tmp$species <- s
        return(tmp)
    }))
    colnames(ratio_bg) <- c("type", "bg", "species")
    
    df <- merge(df, ratio_bg, by = c("species","type"))
    df$type <- factor(df$type, levels = c("WGD", "SSD"))
    p <- ggboxplot(df, x = "type", y = "paralogs.", color = "type", palette = "jco") + 
        stat_summary(fun = "median", geom = "text", aes(label = round(after_stat(y), 3)), vjust = -1) + 
        geom_hline(aes(yintercept = bg, color = type), linetype = "dashed") +
        facet_wrap(~species, nrow = 1) + 
        theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1))
    ggsave(filename = paste0("paralog_ratio.cell_family.summary.", type, ".pdf"),p , width = 4, height = 7)
}


In [12]:
get_ratio(species, "wilcox")